# Importações

In [42]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col, sum, first, concat_ws, unix_timestamp, round, expr, format_string
from pathlib import Path
from delta import configure_spark_with_delta_pip
import os, shutil

Criando a sessão spark

In [2]:
builder = SparkSession.builder \
    .appName("transform-silver") \
    .config("spark.sql.extensions", "delta.sql.DeltaSparkSessionExtensions") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        
spark = configure_spark_with_delta_pip(builder).getOrCreate()

25/05/21 16:24:23 WARN Utils: Your hostname, gabriel-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.16.128 instead (on interface ens33)
25/05/21 16:24:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/gabriel/Documents/PySpark-Projects/Sales-project/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/gabriel/.ivy2/cache
The jars for the packages stored in: /home/gabriel/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-871ce7ef-ddf8-41d7-b00f-b252345b5c90;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.1 in central
	found io.delta#delta-storage;3.3.1 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 2693ms :: artifacts dl 101ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.1 from central in [default]
	io.delta#delta-storage;3.3.1 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |

In [3]:
print(spark)

In [4]:
print(spark.version)

3.5.5


# Variantes

In [5]:
src_path = os.path.expanduser("~/Documents/PySpark-Projects/Sales-project/data/{stage}/{file_name}")

# Transformação tabela FATO

A tabela fato não existe como gostariamos, com isso, vamos estruturar a tabela fato com as informações que temos.

## Lendo arquivo Olist Orders

In [6]:
dataset = spark.read.format('delta').load(src_path.format(stage='silver', file_name='olit_orders'))
dataset.show(5, False)

25/05/21 16:25:31 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|order_id                        |customer_id                     |order_status|order_purchase_timestamp|order_approved_at  |order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b7cc49136f2d6af7|9ef432eb6251297304e76186b10a928d|delivered   |2017-10-02 10:56:33     |2017-10-02 11:07:15|2017-10-04 19:55:00         |2017-10-10 21:25:13          |2017-10-18 00:00:00          |
|53cdb2fc8bc7dce0b6741e2150273451|b0830fb4747a6c6d20dea0b8c802d7ef|delivered   |2018-07-24 20:41:37     |2018-07-26 03:24:27|2018-07-26 14:3

Essta tabela é a base da tabela fato. Vamos agora agregar as informações

## Lendo arquivo Order Payment

In [7]:
dataset_payment = spark.read.format('delta').load(src_path.format(stage='silver', file_name='olit_order_payment'))
dataset_payment.show(5, False)

+--------------------------------+------------------+------------+--------------------+-------------+
|order_id                        |payment_sequential|payment_type|payment_installments|payment_value|
+--------------------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b1e8b2acac839d17|1                 |credit_card |8                   |99.33        |
|a9810da82917af2d9aefd1278f1dcfa0|1                 |credit_card |1                   |24.39        |
|25e8ea4e93396b6fa0d3dd708e76c1bd|1                 |credit_card |1                   |65.71        |
|ba78997921bbcdc1373bb41e913ab953|1                 |credit_card |8                   |107.78       |
|42fdf880ba16b47b59251dd489d4441a|1                 |credit_card |2                   |128.45       |
+--------------------------------+------------------+------------+--------------------+-------------+
only showing top 5 rows



Agora vamos juntar esse dataset com o dataset fato e manter as colunas que são relevantes para nós

In [8]:
dataset = dataset_payment.join(dataset, on="order_id", how="inner")
dataset.show(5, False)

+--------------------------------+------------------+------------+--------------------+-------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|order_id                        |payment_sequential|payment_type|payment_installments|payment_value|customer_id                     |order_status|order_purchase_timestamp|order_approved_at  |order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------------------+------------------+------------+--------------------+-------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b7cc49136f2d6af7|2                 |voucher     |1                   |18.59        |9ef432eb6251297304e76186b10a928d|delivered   |2017-10

In [9]:
dataset = dataset.select(
    col("order_id"),
    col("customer_id"),
    col("payment_type"),
    col("order_status"),
    col("order_purchase_timestamp"),
    col("order_approved_at"),
    col("order_delivered_carrier_date"),
    col("order_delivered_customer_date"),
    col("order_estimated_delivery_date"),
    col("payment_installments"),
    col("payment_sequential"),
    col("payment_value")
)

dataset.show(5, False)

+--------------------------------+--------------------------------+------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+--------------------+------------------+-------------+
|order_id                        |customer_id                     |payment_type|order_status|order_purchase_timestamp|order_approved_at  |order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|payment_installments|payment_sequential|payment_value|
+--------------------------------+--------------------------------+------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+--------------------+------------------+-------------+
|e481f51cbdc54678b7cc49136f2d6af7|9ef432eb6251297304e76186b10a928d|voucher     |delivered   |2017-10-02 10:56:33     |2017-10-02 11:07:15|2017-10-04 19:55

Vamos agora separar Fato x Dimensão de pagamento

Vamos começar com a dimensão de pagamento. A chave única, será a order_id + payment_sequential

In [10]:
dataset_payment = dataset_payment.withColumn('id_payment', concat_ws('_', col('order_id'), col('payment_sequential')))
dataset_payment = dataset_payment.select(
    col('id_payment'),
    col('order_id'),
    col('payment_type'),
    col('payment_installments'),
    col('payment_sequential'),
    col('payment_value')
)
dataset_payment.filter(col('order_id')=='e481f51cbdc54678b7cc49136f2d6af7').show(5, False)

+----------------------------------+--------------------------------+------------+--------------------+------------------+-------------+
|id_payment                        |order_id                        |payment_type|payment_installments|payment_sequential|payment_value|
+----------------------------------+--------------------------------+------------+--------------------+------------------+-------------+
|e481f51cbdc54678b7cc49136f2d6af7_1|e481f51cbdc54678b7cc49136f2d6af7|credit_card |1                   |1                 |18.12        |
|e481f51cbdc54678b7cc49136f2d6af7_3|e481f51cbdc54678b7cc49136f2d6af7|voucher     |1                   |3                 |2.0          |
|e481f51cbdc54678b7cc49136f2d6af7_2|e481f51cbdc54678b7cc49136f2d6af7|voucher     |1                   |2                 |18.59        |
+----------------------------------+--------------------------------+------------+--------------------+------------------+-------------+



vamos agora salvar nossa dimensão de pagamento

In [11]:
Path(src_path.format(stage='gold', file_name='olit_payment_type')).mkdir(parents=True, exist_ok=True)
dataset_payment.write.mode('overwrite')\
    .format('delta')\
        .option("overwriteSchema", "true") \
        .option('compression', 'snappy')\
            .save(src_path.format(stage='gold', file_name='olit_payment_type'))

Agora vamos deixar a fato separada

In [12]:
dataset = dataset.withColumn('payment_id', concat_ws('_', col('order_id'), col('payment_sequential')))
dataset.filter(col('order_id')=='e481f51cbdc54678b7cc49136f2d6af7').show(5, False)

+--------------------------------+--------------------------------+------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+--------------------+------------------+-------------+----------------------------------+
|order_id                        |customer_id                     |payment_type|order_status|order_purchase_timestamp|order_approved_at  |order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|payment_installments|payment_sequential|payment_value|payment_id                        |
+--------------------------------+--------------------------------+------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+--------------------+------------------+-------------+----------------------------------+
|e481f51cbdc54678b7cc49136f2d6af7|9ef432eb6251297

agora temos a fato com dados unicos em nossa coluna chave

## Dimensão de Review

In [13]:
dataset_review=spark.read.format('delta').load(src_path.format(stage='silver', file_name='olit_order_reviews'))
dataset_review.filter(col('order_id')=='e481f51cbdc54678b7cc49136f2d6af7').show(5, False)

+--------------------------------+--------------------------------+------------+--------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+-----------------------+
|order_id                        |review_id                       |review_score|review_comment_title|review_comment_message                                                                                                                                                    |review_creation_date|review_answer_timestamp|
+--------------------------------+--------------------------------+------------+--------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+-----------------------+
|e481f51cbdc54678b7cc49136f2d6af7|a54f0611adc9

Vamos verificar como deve ser a chave dessa coluna

In [14]:
dataset_review.groupBy(
    col('order_id'),
    col('review_id')
).count().filter(col('count')>1).show(truncate=False)

+--------+---------+-----+
|order_id|review_id|count|
+--------+---------+-----+
+--------+---------+-----+



O identificador único deve ser criado baseado na order_id + review_id

Para isso vamos criar a Pk concatenando a order_id + review_id e deixar para o autoincremento carregar a dimensão

In [15]:
dataset_review.show(5, False)

+--------------------------------+--------------------------------+------------+------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+-----------------------+
|order_id                        |review_id                       |review_score|review_comment_title    |review_comment_message                                                                                                                                    |review_creation_date|review_answer_timestamp|
+--------------------------------+--------------------------------+------------+------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+-----------------------+
|729c445f5f350f88d5c9a3911d6e9bc0|8f9aa50ee40780ba3f79614530a34ebe|5           |Mu

Agora vamos salvar a dimensão de reviews na camada gold

In [16]:
Path(src_path.format(stage='gold', file_name='olit_order_reviews')).mkdir(parents=True, exist_ok=True)
dataset_review.write.mode('overwrite')\
    .format('delta')\
        .option('compression', 'snappy')\
            .save(src_path.format(stage='gold', file_name='olit_order_reviews'))

# Trabalhando com a dimensão de Ordens Itens

In [17]:
df_order_itens = spark.read.format('delta').load(src_path.format(stage='silver', file_name='olit_dm_order_itens'))
df_order_itens.filter(col('order_id')=='014405982914c2cde2796ddcf0b8703d').show(5, False)

+--------------------------------+-------------+--------------------------------+--------------------------------+-------------------+-----+-------------+
|order_id                        |order_item_id|product_id                      |seller_id                       |shipping_limit_date|price|freight_value|
+--------------------------------+-------------+--------------------------------+--------------------------------+-------------------+-----+-------------+
|014405982914c2cde2796ddcf0b8703d|1            |6782d593f63105318f46bbf7633279bf|325f3178fb58e2a9778334621eecdbf9|2017-08-01 17:50:17|27.9 |3.81         |
|014405982914c2cde2796ddcf0b8703d|2            |e95ee6822b66ac6058e2e4aff656071a|a17f621c590ea0fab3d5d883e1630ec6|2017-08-01 17:50:17|21.33|25.39        |
+--------------------------------+-------------+--------------------------------+--------------------------------+-------------------+-----+-------------+



Entendendo a regra do negócio, identificamos que uma venda pode ter mais de 1 produto e a soma de todos os produtos deve ser a somoa total da venda sem o frete. Também entendemos que uma venda pode ter mais de um fornecedor. Com isso vamos validar valor total da venda e valor total do frete para levarmos para fato

In [18]:
df_order_itens.groupBy(col('order_id')).count().filter(col('count')>1).show(truncate=False)

+--------------------------------+-----+
|order_id                        |count|
+--------------------------------+-----+
|014405982914c2cde2796ddcf0b8703d|2    |
|05afef1c185862cab9062b322ff25cc5|2    |
|078c2d9bb7e5905e870838448db0d3b3|2    |
|0baa56401ae212e308d73cba2911ae20|2    |
|0cb32545f515420d9f3cba2c55ae3c4f|2    |
|116debdd5705aaa51630c3e292565d06|2    |
|118045506e1c1dda060171af43fe11b4|5    |
|125e0c61fdbfc589f33683322310b7a9|4    |
|1c4a92d82c1b0dec18bef12da3fa7756|3    |
|2f5e702c70321e3f1aea2611b6e240a2|2    |
|354f62589ab0a15eb57e820d6fcbe9b5|2    |
|3cbd4333ba67528bd122a8ad17b73821|2    |
|3e22c141d2861860889375c6fd7b89b3|2    |
|3ee6694bb99d5a6db0deceffcd099597|2    |
|414a0b1cbb58b3ae7ce9c39083fcf007|2    |
|4609e3dffa747068028fe0a5962b41f2|2    |
|46a1ac51daefad9fbb8e65caacd43539|2    |
|482a3e5009a4090348fe6f7d637a53a4|2    |
|483ff5a41552b4071ff9d8bef07c6ee7|2    |
|502d7a6ef832644f5208f0af59e4c5bb|2    |
+--------------------------------+-----+
only showing top

In [19]:
df_order_itens.filter(col('order_id')=='014405982914c2cde2796ddcf0b8703d').show()

+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date|price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|014405982914c2cde...|            1|6782d593f63105318...|325f3178fb58e2a97...|2017-08-01 17:50:17| 27.9|         3.81|
|014405982914c2cde...|            2|e95ee6822b66ac605...|a17f621c590ea0fab...|2017-08-01 17:50:17|21.33|        25.39|
+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+



In [20]:
dataset.filter(col('order_id')=='014405982914c2cde2796ddcf0b8703d').show()

+--------------------+--------------------+------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+--------------------+------------------+-------------+--------------------+
|            order_id|         customer_id|payment_type|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|payment_installments|payment_sequential|payment_value|          payment_id|
+--------------------+--------------------+------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+--------------------+------------------+-------------+--------------------+
|014405982914c2cde...|2de342d6e5905a5a8...| credit_card|   delivered|     2017-07-26 17:38:47|2017-07-26 17:50:17|         2017-07-27 19:39:52|          2017-07-31

vamos unir a fato para trazer as chaves da dimensão de itens

In [21]:
dataset = dataset.join(df_order_itens, on="order_id", how="inner")
dataset = teste.select(
    col("order_id"),
    col('payment_id'),
    col("customer_id"),
    col('seller_id'),
    col('product_id'),
    col("payment_type"),
    col("order_status"),
    col("order_purchase_timestamp"),
    col("order_approved_at"),
    col("order_delivered_carrier_date"),
    col("order_delivered_customer_date"),
    col("order_estimated_delivery_date"),
    col('shipping_limit_date'),
    col("payment_installments"),
    col("payment_sequential"),
    col("payment_value"),
    col('price'),
    col('freight_value')
)
dataset.filter(col('order_id')=='014405982914c2cde2796ddcf0b8703d').show(truncate=False)

NameError: name 'teste' is not defined

Agora nossa fato está com as colunas necessárias e podemos desconsiderar o arquivo de itens como uma dimensão, pois devido a estrutura atual, se adequa mais o seller e o produto serem dimensões separadas

# Dimensão de produtos

In [ ]:
df_produtos = spark.read.format('delta').load(src_path.format(stage='silver', file_name='olist_products'))
df_produtos.filter(col('product_id')=='e95ee6822b66ac6058e2e4aff656071a').show(5, False)

+--------------------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|product_id                      |product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|e95ee6822b66ac6058e2e4aff656071a|esporte_lazer        |52                 |1153                      |1                 |180             |17               |11               |14              |
+--------------------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+



A tabela de produtos está bem estruturada, vamos ver a tipagem das informações

In [ ]:
df_produtos.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (nullable = true)
 |-- product_height_cm: integer (nullable = true)
 |-- product_width_cm: integer (nullable = true)



A tipagem está correta, vamos salvar as informações

In [ ]:
Path(src_path.format(stage='gold', file_name='olit_products')).mkdir(parents=True, exist_ok=True)
df_produtos.write.mode('overwrite')\
    .format('delta')\
        .option('compression', 'snappy')\
            .save(src_path.format(stage='gold', file_name='olit_products'))

Dimensão customers

In [ ]:
df_customer = spark.read.format('delta').load(src_path.format(stage='silver', file_name='olit_dm_customers_order'))
df_customer.filter(col('customer_id')=='2de342d6e5905a5a8bb3a991c855f3e2').show(5, False)

+--------------------------------+--------------------------------+
|customer_id                     |customer_unique_id              |
+--------------------------------+--------------------------------+
|2de342d6e5905a5a8bb3a991c855f3e2|e8d87ee946600f7753579a074fbd2d5d|
+--------------------------------+--------------------------------+



In [ ]:
Path(src_path.format(stage='gold', file_name='olit_dm_customers_order')).mkdir(parents=True, exist_ok=True)
df_customer.write.mode('overwrite')\
    .format('delta')\
        .option('compression', 'snappy')\
            .save(src_path.format(stage='gold', file_name='olit_dm_customers_order'))

In [ ]:
df_customer_unique = spark.read.format('delta').load(src_path.format(stage='silver', file_name='olit_dm_customers'))
df_customer_unique.filter(col('customer_unique_id')=='e8d87ee946600f7753579a074fbd2d5d').show(5, False)

+--------------------------------+------------------------+-------------+--------------+
|customer_unique_id              |customer_zip_code_prefix|customer_city|customer_state|
+--------------------------------+------------------------+-------------+--------------+
|e8d87ee946600f7753579a074fbd2d5d|26551                   |mesquita     |RJ            |
+--------------------------------+------------------------+-------------+--------------+



In [ ]:
Path(src_path.format(stage='gold', file_name='olit_dm_customers')).mkdir(parents=True, exist_ok=True)
df_customer_unique.write.mode('overwrite')\
    .format('delta')\
        .option('compression', 'snappy')\
            .save(src_path.format(stage='gold', file_name='olit_dm_customers'))

# Dimensão de geolocalização

In [ ]:
df_geolocation = spark.read.format('delta').load(src_path.format(stage='silver', file_name='olit_dm_geolocation'))
df_geolocation.show(5)

+---------------------------+-------------------+-------------------+----------------+-----------------+
|geolocation_zip_code_prefix|    geolocation_lat|    geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+-------------------+-------------------+----------------+-----------------+
|                       1001| -23.54929199999999|-46.633559478233785|       sao paulo|               SP|
|                       1002| -23.54831797807146| -46.63542110199666|       sao paulo|               SP|
|                       1003| -23.54903244546711| -46.63531311226845|       sao paulo|               SP|
|                       1004|-23.550115903139222| -46.63512161420169|       sao paulo|               SP|
|                       1005|-23.549819091869107| -46.63560588995324|       sao paulo|               SP|
+---------------------------+-------------------+-------------------+----------------+-----------------+
only showing top 5 rows



Vamos ver a tipagem dos dados

In [ ]:
df_geolocation.printSchema()

root
 |-- geolocation_zip_code_prefix: integer (nullable = true)
 |-- geolocation_lat: double (nullable = true)
 |-- geolocation_lng: double (nullable = true)
 |-- geolocation_city: string (nullable = true)
 |-- geolocation_state: string (nullable = true)



In [ ]:
Path(src_path.format(stage='gold', file_name='olit_geolocation')).mkdir(parents=True, exist_ok=True)
df_geolocation.write.mode('overwrite')\
    .format('delta')\
        .option('compression', 'snappy')\
            .save(src_path.format(stage='gold', file_name='olit_geolocation'))

# Concluindo a fato

In [ ]:
dataset.show(5, False)

+--------------------------------+----------------------------------+--------------------------------+--------------------------------+--------------------------------+------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------------+--------------------+------------------+-------------+-----+-------------+
|order_id                        |payment_id                        |customer_id                     |seller_id                       |product_id                      |payment_type|order_status|order_purchase_timestamp|order_approved_at  |order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|shipping_limit_date|payment_installments|payment_sequential|payment_value|price|freight_value|
+--------------------------------+----------------------------------+--------------------------------+--------------------------------+-------------------

In [24]:
dataset = dataset.select(
    col('order_id'),
    col('payment_id'),
    col('customer_id'),
    col('seller_id'),
    col('product_id'),
    col('order_status'),
    col('order_purchase_timestamp'),
    col('order_approved_at'),
    col('order_delivered_carrier_date'),
    col('order_delivered_customer_date'),
    col('order_estimated_delivery_date'),
    col('shipping_limit_date'),
    col('payment_installments'),
    col('payment_sequential'),
    col('payment_value'),
    col('price'),
    col('freight_value'),

)

dataset.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------------+--------------------+------------------+-------------+-----+-------------+
|            order_id|          payment_id|         customer_id|           seller_id|          product_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|shipping_limit_date|payment_installments|payment_sequential|payment_value|price|freight_value|
+--------------------+--------------------+--------------------+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------------+--------------------+----------

### Agora, vamos realizar cálculos sobre nossa tabela fato, os cáculos serão

- Valor Total do Pedido; <br>
- Tempo de Aprovação do Pedido <br>
- Tempo de Entrega Real <br>
- Atraso na Entrega <br>
- Ticket Médio por Pedido <br>
- Número de Parcelas x Valor por Parcela <br>
- Tempo de Coleta pelo Transportador <br>
- Faturamento por Produto / Pedido / Vendedor <br>
- Margem de Frete <br>
- Qtd. de Parcelas por Pedido <br>
- Status de Entrega Categorizado <br>
- Indicador de Pedido Concluído <br>

#### Valor Total do Pedido <br>
- Ajuda a entender o valor real gerado por cada item (produto + frete), essencial para análises de receita, comportamento de compra e margem de lucro.<br>
- Calculo: (produto + frete)<br>

In [25]:
dataset.show(1)

+--------------------+--------------------+--------------------+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------------+--------------------+------------------+-------------+-----+-------------+
|            order_id|          payment_id|         customer_id|           seller_id|          product_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|shipping_limit_date|payment_installments|payment_sequential|payment_value|price|freight_value|
+--------------------+--------------------+--------------------+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------------+--------------------+----------

In [27]:
dataset = dataset.withColumn('total_order_value', col('price') + col('freight_value'))
dataset.show(3, False)

+--------------------------------+----------------------------------+--------------------------------+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------------+--------------------+------------------+-------------+-----+-------------+-----------------+
|order_id                        |payment_id                        |customer_id                     |seller_id                       |product_id                      |order_status|order_purchase_timestamp|order_approved_at  |order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|shipping_limit_date|payment_installments|payment_sequential|payment_value|price|freight_value|total_order_value|
+--------------------------------+----------------------------------+--------------------------------+--------------------------------+---------

#### Tempo de Aprovação do Pedido
- Permite avaliar o desempenho do processo de aprovação — quanto mais rápido, melhor a experiência do cliente. Pode indicar gargalos no sistema de pagamento.
- Cálculo: order_approved_at - order_purchase_timestamp

In [45]:
dataset = dataset.withColumn(
    'actual_delevery_time_in_number',
    round(
        (unix_timestamp(col('order_approved_at')) - unix_timestamp(col('order_purchase_timestamp'))) / 60, 2
    )
)

dataset = dataset.withColumn(
    'actual_delevery_time',
    format_string(
        '%02d:%02d:%02d',
        ((unix_timestamp(col('order_approved_at')) - unix_timestamp(col('order_purchase_timestamp'))) / 3600).cast('int'),
        (((unix_timestamp(col('order_approved_at')) - unix_timestamp(col('order_purchase_timestamp'))) % 3600) / 60).cast('int'),
        ((unix_timestamp(col('order_approved_at')) - unix_timestamp(col('order_purchase_timestamp'))) % 60).cast('int')
    )
)

dataset.show(3, False)

+--------------------------------+----------------------------------+--------------------------------+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------------+--------------------+------------------+-------------+-----+-------------+-----------------+------------------------------+--------------------+
|order_id                        |payment_id                        |customer_id                     |seller_id                       |product_id                      |order_status|order_purchase_timestamp|order_approved_at  |order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|shipping_limit_date|payment_installments|payment_sequential|payment_value|price|freight_value|total_order_value|actual_delevery_time_in_number|actual_delevery_time|
+--------------------------------+------

#### Tempo de Entrega Real
- Usado para avaliar o SLA (Service Level Agreement) de entrega. Útil para detectar atrasos ou otimizar a logística.
- order_delivered_customer_date - order_purchase_timestamp

In [49]:
dataset = dataset.withColumn(
    'actual_delevery_time_days_in_number',
    round(
        (unix_timestamp(col('order_delivered_customer_date')) - unix_timestamp(col('order_purchase_timestamp'))) / 60, 2
    )
)

dataset = dataset.withColumn(
    'actual_delevery_time_days',
    format_string(
        '%02d:%02d:%02d',
        (((unix_timestamp(col('order_delivered_customer_date')) - unix_timestamp(col('order_purchase_timestamp'))) / 3600)/24).cast('int'), # Dividindo por 24 para ter os dias
        (((unix_timestamp(col('order_delivered_customer_date')) - unix_timestamp(col('order_purchase_timestamp'))) % 3600) / 60).cast('int'),
        ((unix_timestamp(col('order_delivered_customer_date')) - unix_timestamp(col('order_purchase_timestamp'))) % 60).cast('int')
    )
)

dataset.show(3, False)

+--------------------------------+----------------------------------+--------------------------------+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------------+--------------------+------------------+-------------+-----+-------------+-----------------+------------------------------+--------------------+-----------------------------------+-------------------------+
|order_id                        |payment_id                        |customer_id                     |seller_id                       |product_id                      |order_status|order_purchase_timestamp|order_approved_at  |order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|shipping_limit_date|payment_installments|payment_sequential|payment_value|price|freight_value|total_order_value|actual_delevery_time_in_number|

#### Atraso na Entrega
- Permite analisar se o prazo prometido foi cumprido. Se for positivo, houve atraso; se negativo, foi entregue antes do prazo.
- order_delivered_customer_date - order_estimated_delivery_date

In [50]:
dataset = dataset.withColumn(
    'delivery_delay_days_in_number',
    round(
        (unix_timestamp(col('order_delivered_customer_date')) - unix_timestamp(col('order_estimated_delivery_date'))) / 60, 2
    )
)

dataset = dataset.withColumn(
    'delivery_delay_days',
    format_string(
        '%02d:%02d:%02d',
        (((unix_timestamp(col('order_delivered_customer_date')) - unix_timestamp(col('order_estimated_delivery_date'))) / 3600)/24).cast('int'), # Dividindo por 24 para ter os dias
        (((unix_timestamp(col('order_delivered_customer_date')) - unix_timestamp(col('order_estimated_delivery_date'))) % 3600) / 60).cast('int'),
        ((unix_timestamp(col('order_delivered_customer_date')) - unix_timestamp(col('order_estimated_delivery_date'))) % 60).cast('int')
    )
)

dataset.show(3, False)

+--------------------------------+----------------------------------+--------------------------------+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------------+--------------------+------------------+-------------+-----+-------------+-----------------+------------------------------+--------------------+-----------------------------------+-------------------------+-----------------------------+-------------------+
|order_id                        |payment_id                        |customer_id                     |seller_id                       |product_id                      |order_status|order_purchase_timestamp|order_approved_at  |order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|shipping_limit_date|payment_installments|payment_sequential|payment_value|price|freight_value

#### Ticket Médio por Pedido
- Importante para entender o comportamento de compra por cliente, segmentar consumidores e planejar campanhas de marketing.
- Agrupar por order_id e calcular sum(payment_value)